<a href="https://colab.research.google.com/github/binhdodata/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
uploaded = files.upload()


Saving charity_data.csv to charity_data.csv


1. Import Dependencies and Load Data:

In [57]:
# Import dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight

# Load data
data_path = "charity_data.csv"
df = pd.read_csv(data_path)


Preprocess the Data:

In [58]:
# Drop non-beneficial columns
df = df.drop(columns=['EIN', 'NAME'])

# Binning the APPLICATION_TYPE column
application_counts = df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are below a certain threshold
threshold = 500
application_to_replace = list(application_counts[application_counts < threshold].index)

# Replace in dataframe
for app in application_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


Binning the CLASSIFICATION column:


In [59]:
classification_counts = df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are below a certain threshold
threshold = 1500  # You can adjust this threshold based on your understanding of the data
classification_to_replace = list(classification_counts[classification_counts < 15].index)

# Replace in dataframe
for cls in classification_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(df['CLASSIFICATION'].value_counts())


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
Other      143
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
Name: CLASSIFICATION, dtype: int64


In [62]:
# One-Hot Encoding
df_encoded = pd.get_dummies(df)

y = df_encoded["IS_SUCCESSFUL"].values
X = df_encoded.drop(columns=["IS_SUCCESSFUL"]).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handling class imbalance (if any)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

Design Neural Network **Model**

In [66]:

# Initialize the model
nn_optimized = tf.keras.models.Sequential()

# Input layer
input_features = X_train_scaled.shape[1]

# First hidden layer with relu activation
nn_optimized.add(Dense(units=128, input_dim=input_features, activation='relu'))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.3))

# Second hidden layer with relu activation
nn_optimized.add(Dense(units=64, activation='relu'))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.3))

# Third hidden layer with relu activation
nn_optimized.add(Dense(units=32, activation='relu'))
nn_optimized.add(BatchNormalization())
nn_optimized.add(Dropout(0.3))

# Fourth hidden layer with tanh activation
nn_optimized.add(Dense(units=16, activation='tanh'))

# Output layer with sigmoid activation
nn_optimized.add(Dense(units=1, activation='sigmoid'))

# Optimizer with custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

# Compile the model
nn_optimized.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model using test data
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/200
644/644 [==============================] - 10s 12ms/step - loss: 0.6335 - accuracy: 0.6582 - val_loss: 0.5757 - val_accuracy: 0.7141
Epoch 2/200
644/644 [==============================] - 5s 6ms/step - loss: 0.5893 - accuracy: 0.7054 - val_loss: 0.5700 - val_accuracy: 0.7164
Epoch 3/200
644/644 [==============================] - 6s 10ms/step - loss: 0.5771 - accuracy: 0.7183 - val_loss: 0.5672 - val_accuracy: 0.7205
Epoch 4/200
644/644 [==============================] - 5s 7ms/step - loss: 0.5726 - accuracy: 0.7212 - val_loss: 0.5648 - val_accuracy: 0.7242
Epoch 5/200
644/644 [==============================] - 6s 9ms/step - loss: 0.5646 - accuracy: 0.7262 - val_loss: 0.5636 - val_accuracy: 0.7263
Epoch 6/200
644/644 [==============================] - 3s 5ms/step - loss: 0.5640 - accuracy: 0.7285 - val_loss: 0.5630 - val_accuracy: 0.7248
Epoch 7/200
644/644 [==============================] - 3s 5ms/step - loss: 0.5629 - accuracy: 0.7291 - val_loss: 0.5614 - val_accuracy: 0.7

In [37]:
from tensorflow.keras.models import load_model

nn_optimized.save("AlphabetSoupCharity_Optimization.h5")

# Load the model
loaded_nn = load_model("AlphabetSoupCharity_Optimization.h5")

# Use the loaded model as you would the original
loaded_nn.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 128)               5632      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_12 (Dense)            (None, 16)               